# Plot the LSST star catalog

Create alse subsamples of the catalog

### User

In [1]:
#     Patch of the sky to plot and extract

##     The whole sky:

# NOTE: I have to run once the script to determine 'num_of_zeros'
# for a specific range of (ra,dec).


""" 
ra_min = 0.0; ra_max = 360.0;
dec_min = -90.0; dec_max = 90.0;
# 'num_of_zeros' is specific to the given (ra, dec) 
# and does NOT depend on 'JumpsSubsample'.
# 16922944 is the total number of stars in the LSST catalog
num_of_zeros = 16922944
# Jump in the data to create a subsample. Jump size?:
JumpsSubsample = 300; 
# Number of stars to include in the subsample when plotting 
# the data without jumps (see second section of notebook)
num_stars_subsample = 25000 """


#     Patch of the sky observed by OGLE:
ra_min = 250.0; ra_max = 283.0;
dec_min = -40.0; dec_max = -15.0;
# 'num_of_zeros' is specific to the given (ra, dec) 
# and does NOT depend on 'JumpsSubsample'.
num_of_zeros = 6047829 # 
# Jump in the data to create a subsample. Jump size?:
JumpsSubsample = 100; 
# Number of stars to include in the subsample when plotting 
# the data without jumps (see second section of notebook)
num_stars_subsample = 5000

"""
#    Patch of the sky out of the Milky Way buldge
ra_min = 150.0; ra_max = 180.0;
dec_min = 0.0; dec_max = 25.0;
# 'num_of_zeros' is specific to the given (ra, dec) 
# and does NOT depend on 'JumpsSubsample'.
num_of_zeros = 13185 
# Jump in the data to create a subsample. Jump size?:
JumpsSubsample = 10; 
# Number of stars to include in the subsample when plotting 
# the data without jumps (see second section of notebook)
num_stars_subsample = 5000 """

# ---------------------------------------------------

# Create a text file with -ALL- the data that lie within the
# selected patch of the sky?
# Usually I use the option 'False'.
CreateSample = False

# Note a text file will be created by default with the data that 
# lie within the selected patch of the sky but considering only 
# every given data based on "jumps" defined by me.

# ---------------------------------------------------

# Dir of simulated LSST star catalog
DirLSSTStarCatalog = "/Users/arturo/Documents/Research/LSST/Catalog/"

# Dir save output
DirSaveOutput = "/Users/arturo/Documents/Research/LSST/Catalog/Subsamples/"

# -------------------------------
# Given that javascript doesn't work in JupyterLab, I have to set the 
# name of the notebook by hand
NotebookName = 'Plot_LSST_catalog.ipynb'


------

### Automatic

In [2]:
import numpy as np
# import pandas as pd
import os # To use command line like instructions
import glob # To read the files in my directory
from matplotlib import pyplot as plt

5+4

9

#### Useful functions

In [3]:
# Function to identify if a string is an integer number or a letter.
# This will be used in the dictionary construction to properly read some SN names.

def is_number(s):
    try:
        int(s)
        return True
    except ValueError:
        return False

# Tests
print is_number('5'), is_number('e'), is_number(' ')
# True False

True False False


In [4]:
# Get the current date and time
import datetime 

# Read the time and date now
now = datetime.datetime.now()
print now

2018-07-12 11:10:26.972833


------

# Section 1

### Create an array with the stars from a given region in sky

In [5]:
# Initialize the timing of computation
now_start = datetime.datetime.now()
time_start = now_start.strftime("%Y-%m-%d (yyyy-mm-dd); %H:%M hrs.")
print "# Starting time of computing: %s"%time_start

# Starting time of computing: 2018-07-12 (yyyy-mm-dd); 11:10 hrs.


Main loop

In [6]:
#      Main loop

#------------------------
# Initialize array with zeros

# Create the sample text file if desired.
if CreateSample: ra_dec_np = np.zeros([num_of_zeros,2])
else: ra_dec_np = np.zeros([10,2])

# Define the size of the numpy array for the subsample
# create from considering every given data.
num_of_zeros_sub = int(float(num_of_zeros)/float(JumpsSubsample)) + 1 
# Initialize array with zeros:
ra_dec_sub_np = np.zeros([num_of_zeros_sub,2])

#------------------------
# Open to read the main LSST star catalog
LSST_catalog_data = open(DirLSSTStarCatalog+'lsst_star_table.dat', 'r')

#------------------------
# Create a text file to write down the sample.
if CreateSample:
    Sample_file = open(DirSaveOutput+'LSST_ra_%.0f_%.0f_dec_%.0f_%.0f.dat'%(
        ra_min, ra_max, dec_min, dec_max), 'w')
    Sample_file.write("# ra   dec   u   g   r   i   z   y   E(B-V) \n")
    
# Create a text file to write down the subsample.
Subsample_file = open(DirSaveOutput+'LSST_ra_%.0f_%.0f_dec_%.0f_%.0f_Jump_%s.dat'%(
    ra_min, ra_max, dec_min, dec_max, JumpsSubsample), 'w')
Subsample_file.write("# ra   dec   u   g   r   i   z   y   E(B-V) \n")

#------------------------
# Reset counters
count_LCs = 0
count_LCs_sub = 0 # 

for line in LSST_catalog_data:
    
    # Reset variables
    ra = 0.0; dec = 0.0
    # uu=0.0; gg=0.0; rr=0.0; ii=0.0; zz=0.0; yy=0.0;
    
    # Define (ra, dec)
    if is_number(line[9]):
        ra = float(line[:10])
        dec = float(line[11:20])
    else: 
        ra = float(line[:9])
        dec = float(line[10:19])
        
    if ra>ra_min and ra<ra_max and dec>dec_min and dec<dec_max:
        if CreateSample:
            ra_dec_np[count_LCs][0] = ra
            ra_dec_np[count_LCs][1] = dec
            Sample_file.write("%s"%line)
        
        count_LCs += 1
        
        # Create a subarray by saving every given values.
        # The jump between values is set with 'JumpsSubsample'.
        if (float(count_LCs)/float(JumpsSubsample)).is_integer():
            ra_dec_sub_np[count_LCs_sub][0] = ra
            ra_dec_sub_np[count_LCs_sub][1] = dec
            # Write text file:
            Subsample_file.write("%s"%line)
            
            count_LCs_sub += 1
    
#--------------------------

LSST_catalog_data.close();
Subsample_file.close();
if CreateSample: Sample_file.close();

print "# %s stars within (%s<ra<%s, %s<dec<%s)."%(
    count_LCs, ra_min, ra_max, dec_min, dec_max)
print "# %s stars in -subsample- every %s jumps within (%s<ra<%s, %s<dec<%s)."%(
    count_LCs_sub, JumpsSubsample, ra_min, ra_max, dec_min, dec_max)
print "#------------------------"

# 6047829 stars within (250.0<ra<283.0, -40.0<dec<-15.0).
# 60478 stars in -subsample- every 100 jumps within (250.0<ra<283.0, -40.0<dec<-15.0).
#------------------------


In [7]:
# 16922944 stars within (0.0<ra<360.0, -90.0<dec<90.0).
# 56409 stars in -subsample- every 300 jumps within (0.0<ra<360.0, -90.0<dec<90.0).
#------------------------
# 16922944 stars within (0.0<ra<360.0,-90.0<dec<90.0).
# 169229 stars subsample every 100 jumps within (0.0<ra<360.0,-90.0<dec<90.0).
#------------------------
# 13185 stars within (150.0 < ra < 180.0, 0.0 < dec < 25.0).
# 131 stars subsample every 100 jumps within (150.0 < ra < 180.0, 0.0 < dec < 25.0).
#------------------------
# 5850381 stars within (250.0 < ra < 280.0, -40.0 < dec < -15.0).
# 5850381 stars subsample every 1000 jumps within (250.0 < ra < 280.0, -40.0 < dec < -15.0).

In [8]:
# Just in case: close the reading of the LSST catalog file.
# Useful when there are an error during the loop

LSST_catalog_data.close();LSST_catalog_data.close();

Subsample_file.close();Subsample_file.close();
if CreateSample: 
    Sample_file.close();Sample_file.close();

In [9]:
print "#     Time Summary "
print "# Starting time of computing: %s"%time_start
#----------------------
now_end = datetime.datetime.now()
time_end = now_end.strftime("%Y-%m-%d (yyyy-mm-dd); %H:%M hrs.")
print "# Ending time of computing: %s"%time_end
#----------------------
# Compute the time the MCMC computations took:
time_compute = now_end - now_start
print "# Total time of computing:  %s"%time_compute
print "# More than %s iterations in the loop."%count_LCs
print "#---------------------------"

#     Time Summary 
# Starting time of computing: 2018-07-12 (yyyy-mm-dd); 11:10 hrs.
# Ending time of computing: 2018-07-12 (yyyy-mm-dd); 11:11 hrs.
# Total time of computing:  0:00:58.725573
# More than 6047829 iterations in the loop.
#---------------------------


In [10]:
#     Time Summary 
# Starting time of computing: 2018-07-11 (yyyy-mm-dd); 14:33 hrs.
# Ending time of computing: 2018-07-11 (yyyy-mm-dd); 14:34 hrs.
# Total time of computing:  0:00:32.733897
# More than 16922944 iterations in the loop.
#---------------------------

Sanity check the number of data

In [11]:
print "# The first numbers below have to be equal to those reported above, \n\
# specially for subsample."
if CreateSample:
    print "# %s stars within (%s<ra<%s, %s<dec<%s)."%(
        len(ra_dec_np), ra_min, ra_max, dec_min, dec_max)
print "# %s stars in -subsample- every %s jumps within (%s<ra<%s, %s<dec<%s)."%(
    len(ra_dec_sub_np), JumpsSubsample, ra_min, ra_max, dec_min, dec_max)

# The first numbers below have to be equal to those reported above, 
# specially for subsample.
# 60479 stars in -subsample- every 100 jumps within (250.0<ra<283.0, -40.0<dec<-15.0).


In [12]:
# 169230 stars in subsample every 100 jumps within (0.0<ra<360.0, -90.0<dec<90.0).
# 1693 stars subsample every 10000 jumps within (0.0 < ra < 360.0, -90.0 < dec < 90.0).
# 5850381 stars within (250.0 < ra < 280.0, -40.0 < dec < -15.0).

#### Save arrays to a text file (Optional)

In [ ]:
print "# Saving the numpy array: ..."

if CreateSample:
    np.savetxt(DirSaveOutput+"LSST_ra_%.0f_%.0f_dec_%.0f_%.0f_RA_DEC.dat"%(
        ra_min, ra_max, dec_min, dec_max), ra_dec_np, fmt='%8.4f  %9.4f')
np.savetxt(DirSaveOutput+"LSST_ra_%.0f_%.0f_dec_%.0f_%.0f_Jump_%s_RA_DEC.dat"%(
    ra_min, ra_max, dec_min, dec_max, JumpsSubsample), ra_dec_sub_np, fmt='%8.4f  %9.4f')

print "# ... done."

### Plot

In [13]:
# Initialize the timing of computation
now_start = datetime.datetime.now()
time_start = now_start.strftime("%Y-%m-%d (yyyy-mm-dd); %H:%M hrs.")
print "# Starting time of computing: %s"%time_start

# Starting time of computing: 2018-07-12 (yyyy-mm-dd); 11:11 hrs.


In [14]:
#    PLOTTING

# Color points
# Mycolor_points = 'red'
Mycolor_points = 'black'

fig = plt.figure() 

plt.plot(ra_dec_sub_np[:-10,0], ra_dec_sub_np[:-10,1], 
        Mycolor_points, marker='.', ms=1, ls='None', alpha=0.3)

plt.gca().invert_xaxis() # Invert x-axis
plt.xlabel("RA")
plt.ylabel("DEC")
plt.title("LSST star catalog (%s)"%len(ra_dec_sub_np))

plt.savefig(DirSaveOutput+'LSST_ra_%.0f_%.0f_dec_%.0f_%.0f_Jump_%s_Plot.png'%(
    ra_min, ra_max, dec_min, dec_max, JumpsSubsample), dpi=100)
plt.close()


In [15]:
plt.close();plt.close();plt.close();

In [16]:
print "#     Time Summary "
print "# Starting time of computing: %s"%time_start
#----------------------
now_end = datetime.datetime.now()
time_end = now_end.strftime("%Y-%m-%d (yyyy-mm-dd); %H:%M hrs.")
print "# Ending time of computing: %s"%time_end
#---------------------
# Compute the time the MCMC computations took:
time_compute_mcmc = now_end - now_start
print "# Total time of computing:  %s"%time_compute_mcmc
print "# %s data points plotted."%len(ra_dec_sub_np)
print "#---------------------------"

#     Time Summary 
# Starting time of computing: 2018-07-12 (yyyy-mm-dd); 11:11 hrs.
# Ending time of computing: 2018-07-12 (yyyy-mm-dd); 11:11 hrs.
# Total time of computing:  0:00:00.700110
# 60479 data points plotted.
#---------------------------


----------

# Section 2
### Plot the a given portion of the LSST catalog

The key difference with respect to the script above is that now I'll not -jump- every given lines to create a subsample but instead I will use all the rows of data in a given range of rows from the LSST catalog.


In [ ]:
# Initialize the timing of computation
now_start = datetime.datetime.now()
time_start = now_start.strftime("%Y-%m-%d (yyyy-mm-dd); %H:%M hrs.")
print "# Starting time of computing: %s"%time_start

In [ ]:
# Initialize array with zeros

# Define the size of the numpy array for the subsample
# create from considering every given data.
num_of_zeros_sub = num_stars_subsample 
# Initialize array with zeros:
ra_dec_sub_np = np.zeros([num_of_zeros_sub,2])

#------------------------
# Open to read the main LSST star catalog
LSST_catalog_data = open(DirLSSTStarCatalog+'lsst_star_table.txt', 'r')

#------------------------
# Reset counters
count_LCs_sub = 0 # 
count_row =0

for line in LSST_catalog_data:
    
    if count_LCs_sub == num_stars_subsample: break
    
    # Reset variables
    ra = 0.0; dec = 0.0
    
    # Define (ra, dec)
    if is_number(line[9]):
        ra = float(line[:10])
        dec = float(line[11:20])
    else: 
        ra = float(line[:9])
        dec = float(line[10:19])
        
    if ra>ra_min and ra<ra_max and dec>dec_min and dec<dec_max:
        ra_dec_sub_np[count_LCs_sub][0] = ra
        ra_dec_sub_np[count_LCs_sub][1] = dec
        count_LCs_sub += 1
        
    count_row += 1
     
LSST_catalog_data.close();

print "# %s stars in -subsample- in the first %s rows within (%s<ra<%s, %s<dec<%s)."%(
    count_LCs_sub, count_row, ra_min, ra_max, dec_min, dec_max)
print "#------------------------"

In [ ]:
# 100 stars in -subsample- in the first 100 rows within (0.0<ra<360.0, -90.0<dec<90.0).
#------------------------

In [ ]:
# Just in case: close the reading of the LSST catalog file.
LSST_catalog_data.close();LSST_catalog_data.close();

In [ ]:
print "#     Time Summary "
print "# Starting time of computing: %s"%time_start
#----------------------
now_end = datetime.datetime.now()
time_end = now_end.strftime("%Y-%m-%d (yyyy-mm-dd); %H:%M hrs.")
print "# Ending time of computing: %s"%time_end
#----------------------
# Compute the time the MCMC computations took:
time_compute = now_end - now_start
print "# Total time of computing:  %s"%time_compute
print "# %s iterations in the loop."%count_row
print "#---------------------------"

#### (Optional) Save arrays to a text file

In [ ]:
print "# Saving the numpy array: ..."

np.savetxt(DirSaveOutput+"LSST__ra_%.0f_%.0f_dec_%.0f_%.0f_Stars_%s.dat"%(
    ra_min, ra_max, dec_min, dec_max, num_stars_subsample), 
    ra_dec_sub_np, fmt='%8.4f  %9.4f')

print "# ... done."

### Plot
This subsection is a copy/paste of the plotting section above

In [ ]:
# Initialize the timing of computation
now_start = datetime.datetime.now()
time_start = now_start.strftime("%Y-%m-%d (yyyy-mm-dd); %H:%M hrs.")
print "# Starting time of computing: %s"%time_start

In [ ]:
#    PLOTTING

fig = plt.figure() 

plt.plot(ra_dec_sub_np[:-10,0], ra_dec_sub_np[:-10,1], 
        'black', marker='.', ms=1, ls='None', alpha=0.3)

plt.xlabel("RA")
plt.ylabel("DEC")
plt.title("LSST star catalog (%s)"%len(ra_dec_sub_np))

plt.savefig(DirSaveOutput+'LSST__ra_%.0f_%.0f_dec_%.0f_%.0f_Stars_%s_Plot.png'%(
    ra_min, ra_max, dec_min, dec_max, num_stars_subsample), dpi=100)
plt.close()

In [ ]:
plt.close();plt.close();plt.close();

In [ ]:
print "#     Time Summary "
print "# Starting time of computing: %s"%time_start
#----------------------
now_end = datetime.datetime.now()
time_end = now_end.strftime("%Y-%m-%d (yyyy-mm-dd); %H:%M hrs.")
print "# Ending time of computing: %s"%time_end
#----------------------
# Compute the time the MCMC computations took:
time_compute_mcmc = now_end - now_start
print "# Total time of computing:  %s"%time_compute_mcmc
print "# %s data points plotted."%len(ra_dec_sub_np)
print "#---------------------------"

----------

# Section 3
# Determine the number of stars brighter/dimmer than a given magnitude from the LSST catalog

### User

In [17]:
# Apparent Magnitude limit
# must be a float number
appmag_lim = 21.0

# Band: i

#### Automatic

#### Upload the LSST data from a subsample file

In [18]:
# Initialize the timing of computation
now_start = datetime.datetime.now()
time_start = now_start.strftime("%Y-%m-%d (yyyy-mm-dd); %H:%M hrs.")
print "# Starting time of computing: %s"%time_start

# Starting time of computing: 2018-07-12 (yyyy-mm-dd); 11:11 hrs.


In [19]:
# i band

LSST_sample_filename = 'LSST_ra_250_283_dec_-40_-15.dat'
LSST_data = np.genfromtxt(DirSaveOutput+LSST_sample_filename, usecols=[5])

IOError: /Users/arturo/Documents/Research/LSST/Catalog/Subsamples/LSST_ra_250_283_dec_-40_-15.dat not found.

In [ ]:
print "#     Time Summary "
print "# Starting time of computing: %s"%time_start
#----------------------
now_end = datetime.datetime.now()
time_end = now_end.strftime("%Y-%m-%d (yyyy-mm-dd); %H:%M hrs.")
print "# Ending time of computing: %s"%time_end
#----------------------
# Compute the time the MCMC computations took:
time_compute = now_end - now_start
print "# Total time of computing:  %s"%time_compute
print "#---------------------------"

In [ ]:
len(LSST_data)

In [ ]:
print max(LSST_data), min(LSST_data)

#### Loop

In [ ]:
# Initialize the timing of computation
now_start = datetime.datetime.now()
time_start = now_start.strftime("%Y-%m-%d (yyyy-mm-dd); %H:%M hrs.")
print "# Starting time of computing: %s"%time_start

In [ ]:
# MAIN LOOP

count_dimmer = 0;
count_brighter = 0;

for i in range(len(LSST_data)):
    if LSST_data[i] < appmag_lim:
        count_brighter += 1
    else: count_dimmer += 1

print "# Loop finished."

In [ ]:
print "#     Time Summary "
print "# Starting time of computing: %s"%time_start
#----------------------
now_end = datetime.datetime.now()
time_end = now_end.strftime("%Y-%m-%d (yyyy-mm-dd); %H:%M hrs.")
print "# Ending time of computing: %s"%time_end
#----------------------
# Compute the time the MCMC computations took:
time_compute = now_end - now_start
print "# Total time of computing:  %s"%time_compute
print "# %s iterations in the loop."%len(LSST_data)
print "#---------------------------"

#### Report

Write report to a text file

In [ ]:
total_count = count_brighter+count_dimmer

percentage_brigther = (float(count_brighter)*float(100))/float(total_count)
percentage_dimmer = (float(count_dimmer)*float(100))/float(total_count)

#-------------------------------------------------
# if the report file exist, then open in -append- mode
try: 
    textfile_2 = open(DirSaveOutput+LSST_sample_filename[:-4]+'_Report.dat', 'a')
# otherwise, create it
except:
    textfile_2 = open(DirSaveOutput+LSST_sample_filename[:-4]+'_Report.dat', 'w')
#-------------------------------------------------

text_octothorp = '\n#'+'#'*65 + '\n'
textfile_2.write(text_octothorp); 

textfile_2.write("\n#    Report of brighter/dimmer stars than %s mag in the LSST catalog \n\
# For a given range of (ra, dec).\n"%appmag_lim)

now = datetime.datetime.now() # Read the time and date right now
text_timenow = now.strftime("%Y-%m-%d (yyyy-mm-dd); %H:%M hrs.")
text_Date   = '# On date: %s \n'%text_timenow
text_Author = '# Data table created by: Arturo Avelino \n'
text_script = '# Script used: %s \n'%NotebookName
text_line = '#'+'-'*45 + '\n'

textfile_2.write(text_line); 
textfile_2.write(text_Author); textfile_2.write(text_Date); textfile_2.write(text_script);
textfile_2.write(text_line); 

text_02 = "# In the '%s' subsample file, \n# for i-band:\n"%LSST_sample_filename
text_03 = "%s # stars (%.3f percent) with app mag < %s mag.\n"%(count_brighter, percentage_brigther, appmag_lim)
text_04 = "%s # stars (%.3f percent) with app mag > %s mag.\n"%(count_dimmer, percentage_dimmer, appmag_lim)
text_05 = "%s # = %s = total number of stars.\n"%(total_count, len(LSST_data))
text_06 = "%.4f #  mag is the brigthest star in the given region of the catalog.\n"%min(LSST_data)
text_07 = "%.4f # mag is the dimmest star in the given region of the catalog.\n"%max(LSST_data)

textfile_2.write(text_02); textfile_2.write(text_03); textfile_2.write(text_04);
textfile_2.write(text_05); textfile_2.write(text_06); textfile_2.write(text_07);

print text_02,text_03,text_04,text_05,text_06,text_07

textfile_2.close()

In [ ]:
textfile_2.close(); textfile_2.close(); 

----

# Section 4

## Some diverse simple computations

In [42]:
total_stars = 6047829 # = 6047829 = total number of stars.
brighter_21 = 822641 # stars (13.602 percent) with app mag < 21.0 mag.
brighter_24 = 3038290 # stars (50.238 percent) with app mag < 24.0 mag.
brighter_26 = 5976702 # stars (98.824 percent) with app mag < 26.0 mag.

In [60]:
# Number of stars with 21 < app mag < 24 mag.
print (brighter_24 - brighter_21)

# Percentage
print float((brighter_24 - brighter_21)*100)/float(total_stars), '%'


2215649
36.6354438923 %


In [61]:
50.238 - 13.602

36.635999999999996